# Kagle inclass https://www.kaggle.com/c/simplesentiment/overview

In [1]:
import os
from pathlib import Path

import pandas as pd
import numpy as np

from itertools import product
import warnings
from tqdm import tqdm

import re

In [2]:
import nltk
#nltk.download('movie_reviews')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('rslp')

import nltk.stem as st

In [3]:
import spacy

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.sklearn_api import W2VTransformer

unable to import 'smart_open.gcs', disabling that module


In [76]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

#from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV, cross_val_score 

#from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [45]:
import xgboost as xgb
from lightgbm import LGBMClassifier

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
PATH_DATA = os.path.join(Path.cwd(), 'data')
PATH_SUBM = os.path.join(Path.cwd(), 'submissions')

In [8]:
df = pd.read_csv(os.path.join(PATH_DATA, 'products_sentiment_train.tsv'), 
                    header = None, 
                    index_col = None,
                    sep = '\t',
                   )
df.columns = ['text', 'target']
df.shape

(2000, 2)

In [9]:
df.head()

,text,target
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [10]:
df.target.value_counts()

1    1274
0     726
Name: target, dtype: int64

Классы явно не сбалансированы.   
Т.к. мы работаем с текстами есть следующие варианты:   
    1. добавить негативные примеры. + увеличение выборки, баланс классов. - необходимо найти размеченную, подходящую под тематику, выборку   
    2. дублировать некоторые строки, например те, в которых ошибается модель   
    3. настроить параметр class_weights
    

### Приводим к строчным (на всякий) и очищаем по порядку:

In [11]:
# - все спецсимволы
# - все цифры
# - заменяем множественные пробелы на единичные

In [12]:
clean_text = lambda x: re.sub(r"\s+", ' ', 
                              re.sub(r"[\d+]", '',
                                     re.sub(r"[^\w\s]", '', x.lower()).strip()
                                    )
                             )

In [13]:
df['text_cl'] = df.text.map(clean_text)

In [14]:
#tagged_data[:5]

In [15]:
#lem_text("don't"), lem_text("i'll")

In [16]:
#stem_text("don't"), stem_text("i'll")

### Лемматизация & стемминг

In [17]:
lemm = st.WordNetLemmatizer()
lem_text = lambda x: ' '.join([lemm.lemmatize(el) for el in x.split()])


stemm = st.RSLPStemmer()
stem_text = lambda x: ' '.join([stemm.stem(el) for el in x.split()])

In [18]:
df['text_cl'] = df.text.map(clean_text)
df['text_cl'] = df.text_cl.map(lem_text)
df['text_cl'] = df.text_cl.map(stem_text)

In [19]:
df['text_cl'] = df.text_cl.map(lambda x: x.split())

In [20]:
df.head()

,text,target,text_cl
0,"2 . take around 10,000 640x480 pictures .",1,"[tak, around, x, pictur]"
1,i downloaded a trial version of computer assoc...,1,"[i, downloaded, a, trial, version, of, comput,..."
2,the wrt54g plus the hga7t is a perfect solutio...,1,"[the, wrtg, plu, the, hgat, is, a, perfect, so..."
3,i dont especially like how music files are uns...,0,"[i, dont, especially, lik, how, music, fil, ar..."
4,i was using the cheapie pail ... and it worked...,1,"[i, wa, using, the, cheapi, pail, and, it, wor..."


### Подготавливаем к обучению по сетке

In [21]:
train, test, train_target, test_target = train_test_split(df.text_cl, df.target, 
                                                          test_size = 0.2, 
                                                          stratify = df.target, 
                                                          random_state = 52138,
                                                         )

### Параметры для перебора

In [24]:
pipe_cnt = Pipeline([#('w2v', W2VTransformer(size=100, min_count=1, seed=1)),
                     ('clf', LogisticRegression()),
                      ])

In [25]:
parameters_v0 = {
        'clf': [LogisticRegression(), 
                LinearSVC(),
                #SGDClassifier(), 
                #RandomForestClassifier(),
                #xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0),
                #xgb.XGBClassifier(eval_metric = 'auc'),
               ],
    }

In [60]:
parameters = [
            {'clf': [LogisticRegression()]}, 
            {'clf': [LinearSVC(max_iter = 1500)]},
            #{'clf': [SGDClassifier()]}, 
            #{'clf': [RandomForestClassifier()]},
            #{'clf': [xgb.XGBClassifier(eval_metric = 'auc', use_label_encoder=False)]},
                     #xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0),
            #{'clf': [LGBMClassifier()]},
]

In [63]:
par_size = [300, 200,]# 100, 50]#10, 20, 30, 50, ]
par_window = [5,]# 4, 3,]# 2]
#par_hs = [1,]# 0]
par_niter = [50, 40,]# 30,]# 75,]# 100]#3, 5, 10, 50]
par_min_count = [5, 3,]# 10, 2, 1]
len(list(product(par_size, 
                 par_window, 
                 #par_hs, 
                 par_niter, 
                 par_min_count)))

8

In [97]:
# Трансфорование данных (отзыва) в вектор
# требуется исключить слова, которые не встречались, т.к. выдаст ошибку
# и полученные вектораотзыва усредняем по столбцам
def transform_x2v(inp_model, inp_series):
    transformed_ndarray = np.ndarray((inp_series.shape[0], inp_model.gensim_model.vector_size))
    
    for row, idx in enumerate(inp_series.index):
        
        tr_string = ''
        for wrd in inp_series[idx]:
            if wrd in inp_model.gensim_model.wv.vocab:
                tr_string = ' '.join([tr_string, wrd])

        transformed_ndarray[row] = np.mean(inp_model.transform(tr_string.split()), axis = 0)
    
    return transformed_ndarray

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [68]:
%%time
warnings.filterwarnings("ignore")


scores = [('model', (0, 0, 0, 0), 0) for idx in range(5)]
for (sz, wind, it, mc) in tqdm(product(par_size, par_window, par_niter, par_min_count), 
                              total = len(list(product(par_size, par_window, par_niter, par_min_count)))):
    model_x2v = W2VTransformer(size=sz, window = wind, hs = 1, iter = it, min_count = mc,
                               seed = 16544873
                              )
    #model_x2v.fit(train)
    model_x2v.fit(df.text_cl)
    #train_x2v = transform_x2v(model_x2v, train)
    train_x2v = np.nan_to_num(transform_x2v(model_x2v, train))

    grid = GridSearchCV(pipe_cnt, parameters, cv = 5, scoring = 'roc_auc', verbose = 0, n_jobs=-1)
    ret = grid.fit(train_x2v, train_target)
    if grid.best_score_ > scores[4][2]:
        scores[4] = (grid.best_estimator_['clf'].__class__, (sz, wind, it, mc), grid.best_score_)
        scores.sort(key=lambda tup: tup[2], reverse = True)



#warnings.filterwarnings("always")
warnings.filterwarnings("default")

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:35<00:00,  4.48s/it]

Wall time: 35.8 s


Посмотрим на несколько лучших результатов

In [69]:
scores

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(sklearn.svm._classes.LinearSVC, (200, 5, 50, 5), 0.8453186181634458),
 (sklearn.svm._classes.LinearSVC, (300, 5, 50, 5), 0.8447524810100875),
 (sklearn.svm._classes.LinearSVC, (200, 5, 40, 5), 0.8438951796456868),
 (sklearn.svm._classes.LinearSVC, (300, 5, 40, 5), 0.8374836539390291),
 (sklearn.svm._classes.LinearSVC, (300, 5, 50, 3), 0.8334031755938449)]

Промежуточные выводы /параметры для W2VTransformer:
- размер embedded вектора - 200
- max расстояние между словами всегда 5
- hs всегда 1
- кол-во итерций построения словаря - 50
- модель - LinearSVC


### Попробуем определить причины ошибок по тесту

In [70]:
model_x2v = W2VTransformer(size=200, window = 5, hs = 1, iter = 50, min_count = 5,
                           seed = 16544873
                          )
#model_x2v.fit(train)
model_x2v.fit(df.text_cl)
#train_x2v = transform_x2v(model_x2v, train)
train_x2v = np.nan_to_num(transform_x2v(model_x2v, train))

model = LinearSVC(max_iter = 2500)
model.fit(train_x2v, train_target)

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\_v010ch_\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\_v010ch_\Anaconda3\lib\site-packages\numpy\core\_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


LinearSVC(max_iter=1500)

In [73]:
test_pred = model.predict(np.nan_to_num(transform_x2v(model_x2v, test)))

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [74]:
confusion_matrix(test_target, test_pred)

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[ 81,  64],
       [ 45, 210]], dtype=int64)

только половина отрицательных отзывов, посмотрим их

In [75]:
for idx in range(test.shape[0]):
    if test_pred[idx] != test_target[test.index[idx]]:
        print(test.index[idx],test_target[test.index[idx]], df.loc[test.index[idx]].text)

328 0 ( i never know when the champ is full as there is no way too tell . even though not particularly aesthetic , how about making the champ clear and see-through ? ) so , i lift the lid and find a wad of diaper , which i must remove , usually stuck inside the lid itself because the bag is too full .
1736 0 it is slightly larger than an ipod , no doubt about it . 
1650 0 the gprs web access is sloooooooooooooow !
1638 0 i know this unit will have problems in the future and the $ 40 for the 2 year " no questions asked " replacement warranty will be worth it , since creative will only warrant it for 90 days - which in itself is totally ridiculous ! 
931 1 it is the most bang-for-the-buck out there . 
315 1 norton pops up a window about the intrusions and gives me as much detail as possible about where the intrusion is coming from .
736 0 while the software may not be as easy as i-tunes , it is not difficult to use by any stretch .
1972 1 my first impression was positive . 
1777 0 it als

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Если посмотреть на отзывы, в которых модель ошиблась - то в них нет ничего необычного.   
Большая часть однозначно определяется как положительные или отрицательные.   
Предполагаю, что влияют 2 фактора: несбалансированность класса и не больша выборка для обучения
Попробуем исправить, настроив class_weights

# Пробуем настроить параметры алгоритма

#### в 3-4 итерации. в начале ищем приближенные лучшие значения(закомментированные параметры ниже),   
#### потом их уточняем

In [87]:
pipe_cnt = Pipeline([('clf', SVC()),
                      ])

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [115]:
model_parameters = {
    'clf__max_iter' : [2500, 2000],# 1500],
    'clf__kernel': ['linear'],# 'poly', 'rbf', 'sigmoid',],
    #'clf__C': np.logspace(-2, 2, 10),
    'clf__C': np.linspace(1.7, 2, 40),
    #'clf__class_weight': [{0: 0.5 - shift, 1:0.5 + shift} for shift in np.linspace(-0.1, 0.1, 20)],# + ['balanced'],
    'clf__class_weight': [{0: 0.56 - shift, 1:0.44 + shift} for shift in np.linspace(-0.01, 0.01, 20)]# + ['balanced'],
}

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [133]:
%%time

model_x2v = W2VTransformer(size=200, window = 5, hs = 1, iter = 50, min_count = 5,
                           seed = 16544873
                          )
model_x2v.fit(df.text_cl)
train_x2v = np.nan_to_num(transform_x2v(model_x2v, train))

grid_tune = GridSearchCV(pipe_cnt, model_parameters, cv = 5, scoring = 'roc_auc', verbose = 1, n_jobs=-1)
grid_tune.fit(train_x2v, train_target)
grid_tune.best_estimator_

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\_v010ch_\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\_v010ch_\Anaconda3\lib\site-packages\numpy\core\_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


Fitting 5 folds for each of 3200 candidates, totalling 16000 fits
Wall time: 27min 31s


C:\Users\_v010ch_\Anaconda3\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Pipeline(steps=[('clf',
                 SVC(C=2.41025641025641,
                     class_weight={0: 0.5246153846153846,
                                   1: 0.47538461538461535},
                     kernel='linear', max_iter=2000))])

In [145]:
grid_tune.best_params_

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'clf__C': 2.41025641025641,
 'clf__class_weight': {0: 0.5246153846153846, 1: 0.47538461538461535},
 'clf__kernel': 'linear',
 'clf__max_iter': 2000}

In [146]:
grid_tune.best_score_

0.8421785955761616

Посмотрим на несколько лучших результатов, после чего уточняем параметры и повторяем (несколько раз) GridSearchCV

In [136]:
interested_tune = np.where(grid_tune.cv_results_['rank_test_score'] <= 5)
for field in model_parameters.keys():
    print(field)
    for idx in interested_tune:
        print(grid_tune.cv_results_['param_' + field][idx])

clf__max_iter
[2000 2000 2000 2000 2500]
clf__kernel
['linear' 'linear' 'linear' 'linear' 'linear']
clf__C
[2.217948717948718 2.282051282051282 2.3333333333333335 2.41025641025641
 2.4743589743589745]
clf__class_weight
[{0: 0.538974358974359, 1: 0.461025641025641}
 {0: 0.5287179487179487, 1: 0.47128205128205125}
 {0: 0.5317948717948718, 1: 0.46820512820512816}
 {0: 0.5246153846153846, 1: 0.47538461538461535}
 {0: 0.5333333333333333, 1: 0.4666666666666666}]


C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [137]:
grid_tune.cv_results_['mean_test_score'][np.where(grid_tune.cv_results_['rank_test_score'] <= 5)]

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([0.84135114, 0.84157144, 0.84194523, 0.8421786 , 0.8413351 ])

Остановимся на следующих параметрах:    
    
SVC
kernel = 'linear'
C =  2.41025641025641
сlass_weight = {0: 0.5246153846153846, 1: 0.47538461538461535}
max_iter = 2000

Получим модель и submission для выбранных параметров

### Подготавливаем данные для обучения финальной модели и прогноза

In [147]:
df_test = pd.read_csv(os.path.join(PATH_DATA, 'products_sentiment_test.tsv'),
                        index_col = None,
                        sep = '\t',
                     )
df_test.shape

(500, 2)

In [148]:
#df_test.head(), df_test.tail()

In [149]:
subm = pd.read_csv(os.path.join(PATH_DATA, 'products_sentiment_sample_submission.csv'))
subm.shape

(500, 2)

In [150]:
#subm.head(), subm.tail()

In [151]:
test_df = df_test.text.map(clean_text)
test_df = test_df.map(lem_text)
test_df = test_df.map(stem_text)
test_df = test_df.map(lambda x: x.split())

In [152]:
test_x2v = np.nan_to_num(transform_x2v(model_x2v, test_df))

### Финальная модель для прогноза и прогноз

In [153]:
pred = grid_tune.best_estimator_.predict(test_x2v)
subm.y = pred

In [154]:
subm.to_csv(os.path.join(PATH_SUBM, 'gensim_w2v_own_lsvc_tuned.csv'), index = False)

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


pb 0.74000